# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, we use the tools we learned about in the lesson to identify lane lines on the road.  We develop a pipeline on a series of individual images, and later apply the result to a video stream.

The pipeline can be described as follows:

1. Input image is converted to grayscale.
2. Two color masks are computed and combined, for white and yellow color.
3. The color masks are applied to clear all colors outside the selected color ranges.
4. Gaussian smoothing is applied.
5. Canny edge detection is applied to reveal edges.
6. Hough transform is applied to convert the points in the image to lines.
7. The lines are averaged and extended to their extreme points to output two single lines for left and right.
8. The averaged lines are smoothed based on their endpoints using moving average.
9. The lines are drawn on top of the original color image (annotation).
10. The results are saved to relevant files.
---

## Import Packages

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os
import math

%matplotlib inline

is_first_frame = False

avgLeft = (0, 0, 0, 0)
avgRight = (0, 0, 0, 0)

## Helper Functions

In [2]:
def movingAverage(avg, new_sample, N=20):
    """
    Used to eliminate jitter for the final left, right lines
    produced by the pipeline. Uses the global variables avgLeft,
    avgRight, which represent the endpoints for the left and 
    right lines
    """
    if (avg == 0):
        return new_sample
    avg -= avg / N;
    avg += new_sample / N;
    return avg;

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, upper_boundary, color=[255, 0, 0], thickness=12):
    """
    Draws the lines resulted from the Hough tranform, annotating the original image
    """
    
    # Tries to identify if we process the first frame of the video stream in
    # order to set initial conditions for the moving average used for smoothing
    global is_first_frame

    # Our goal is to classify the lines in set of lines belonging to the left and 
    # right group, where the medium is an imaginary line that splits the image
    # vertically in two halves
    h, w, ch = img.shape

    # We will find the average values for all endpoints of all lines 
    # for each group of lines belonging to the left or right group
    left_y_avg = 0
    left_x_avg = 0
    left_slope_avg = 0
    left_lines_count = 0

    right_y_avg = 0
    right_x_avg = 0
    right_slope_avg = 0
    right_lines_count = 0

    for line in lines:
        for x1, y1, x2, y2 in line:

            # I 'd rather work with cartesian than with image coordinates
            y1 = h - y1
            y2 = h - y2

            # Find the slope of current line
            slope = (y2 - y1) / (x2 - x1)

            # If line is nearly horizontal, a division by zero might occur
            # Discard it
            if (slope<0.1) and (slope>-0.1):
                continue

            # Since we inverted the vertical axis
            # positive slope means left line group
            # Checks out also the position relative to
            # the imaginary verical line splitting vertically the image
            # in two halves
            if ((slope) > 0) and (x1 < (w / 2)):        # left
                left_slope_avg = left_slope_avg + slope
                left_y_avg = left_y_avg + ((y2 + y1) / 2)
                left_x_avg = left_x_avg + ((x2 + x1) / 2)
                left_lines_count = left_lines_count + 1
            else:
                right_slope_avg = right_slope_avg + slope
                right_y_avg = right_y_avg + ((y2 + y1) / 2)
                right_x_avg = right_x_avg + ((x2 + x1) / 2)
                right_lines_count = right_lines_count + 1

    # Found some lines for left
    if (left_lines_count > 0):
        # Found average values for all endpoints. 
        # Essentially this defines a point in the final line we 
        # want to draw: (x0, y0)
        left_y_avg = (left_y_avg) / left_lines_count
        left_x_avg = left_x_avg / left_lines_count
        # Found the average slope, essentially the slope of the
        # final line we want to draw
        left_slope_avg = left_slope_avg / left_lines_count
        # y0 = slope * x0 + beta => beta = (y0 - slope * x0) 
        left_beta = left_y_avg - (left_slope_avg * left_x_avg)
        
        # Solving the line equation dor x to find lower and upper values
        left_x_bottom = (0 - left_beta) / left_slope_avg
        left_x_top = ((h-upper_boundary) - left_beta) / left_slope_avg

        global avgLeft

        if is_first_frame:
            # sets initial condition for moving average
            avgx1, avgy1, avgx2, avgy2 = left_x_bottom, h, left_x_top, upper_boundary
        else:
            avgx1, avgy1, avgx2, avgy2 = avgLeft

        # for the set of all images in a video
        # compute an averaged line for every single frame
        avgLeft = (movingAverage(avgx1, left_x_bottom),
                   movingAverage(avgy1, h),
                   movingAverage(avgx2, left_x_top),
                   movingAverage(avgy2, upper_boundary))
        avgx1, avgy1, avgx2, avgy2 = avgLeft

        # draw the line
        cv2.line(img, (int(avgx1), int(avgy1)), (int(avgx2), int(avgy2)), color, thickness)  # draw left line

    # for right line same as y
    if (right_lines_count > 0):
        right_y_avg = right_y_avg / right_lines_count
        right_x_avg = right_x_avg / right_lines_count
        right_slope_avg = right_slope_avg / right_lines_count
        right_beta = right_y_avg - (right_slope_avg * right_x_avg)
        right_x_bottom = (0 - right_beta) / right_slope_avg
        right_x_top = ((h-upper_boundary) - right_beta) / right_slope_avg

        global avgRight

        if is_first_frame:
            avgx1, avgy1, avgx2, avgy2 = right_x_bottom, h, right_x_top, upper_boundary
        else:
            avgx1, avgy1, avgx2, avgy2 = avgRight

        avgRight = (movingAverage(avgx1, right_x_bottom),
                    movingAverage(avgy1, h),
                    movingAverage(avgx2, right_x_top),
                    movingAverage(avgy2, upper_boundary))
        avgx1, avgy1, avgx2, avgy2 = avgRight

        cv2.line(img, (int(avgx1), int(avgy1)), (int(avgx2), int(avgy2)), color, thickness)  # draw left line

    is_first_frame = False

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, upper_boundary):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                            maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, upper_boundary)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [3]:
def process_image(image):
    # Grayscale the image and define a white mask
    gray = grayscale(image)
    mask_white = cv2.inRange(gray, 200, 255)
    
    # Use the hsv = [hue, saturation, value] representation of the image
    # which allows to select accurate range for yellow
    # Proper values for range found in google
    # Define a yellow mask
    image_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_yellow = np.array([20, 100, 100], dtype="uint8")
    upper_yellow = np.array([30, 255, 255], dtype="uint8")
    mask_yellow = cv2.inRange(image_hsv, lower_yellow, upper_yellow)
    
    # Retrieve the final masked image
    mask_yellow_white = cv2.bitwise_or(mask_white, mask_yellow)
    mask_yellow_white_image = cv2.bitwise_and(gray, mask_yellow_white)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(mask_yellow_white_image, kernel_size)

    # Define our parameters for Canny and apply
    # Used the same parameters as in exercises
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Define mask
    # Using the following values for the mask, led to perfect annotations in static images
    # lower_left = (140, 540)
    # top_left = (435, 330)
    # top_right = (535, 330)
    # lower_right = (900, 540)

    # The videos needed a variable definition for the mask though...
    image_shape = image.shape

    # Define the upper value (h-y) for the defined mask
    # The lower boundary is h
    upper_boundary = image_shape[0]/2 + image_shape[0]/8.5

    lower_left = [image_shape[1]/9, image_shape[0]]
    lower_right = [image_shape[1]-image_shape[1]/9, image_shape[0]]
    top_left = [image_shape[1]/2-image_shape[1]/8, upper_boundary]
    top_right = [image_shape[1]/2+image_shape[1]/8, upper_boundary]
    vertices = [np.array([lower_left, top_left, top_right, lower_right], dtype=np.int32)]
    masked_edges = region_of_interest(edges, vertices)

    # Define the Hough transform parameters
    # Those values are significaly different from the values used in the exercises
    rho = 4                 # distance resolution in pixels of the Hough grid
    theta = np.pi/180       # angular resolution in radians of the Hough grid
    threshold = 30          # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100   # minimum number of pixels making up a line
    max_line_gap = 180      # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    line_image = hough_lines(masked_edges, rho, theta, threshold,
                             min_line_length, max_line_gap, upper_boundary)

    # Draw the lines on the input image
    annotated_image = weighted_img(line_image, image, 0.8, 1, 0)

    return annotated_image

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
for image_name in os.listdir("test_images/"):
    image = mpimg.imread('test_images/' + image_name)
    image_processed = process_image(image)
    mpimg.imsave("annotated_images/annotated_" + image_name, image_processed)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

Let's try the one with the solid white lane on the right first ...

In [5]:
# Tries to identify if we process the first frame of the video stream in
# order to set initial conditions for the moving average used for smoothing
is_first_frame = True

# Execute the pipeline for the video file
white_output = 'annotated_videos/white.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video annotated_videos/white.mp4
[MoviePy] Writing video annotated_videos/white.mp4


100%|█████████▉| 221/222 [00:05<00:00, 36.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: annotated_videos/white.mp4 

Wall time: 6.73 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [7]:
is_first_frame = True
yellow_output = 'annotated_videos/yellow.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video annotated_videos/yellow.mp4
[MoviePy] Writing video annotated_videos/yellow.mp4


100%|█████████▉| 681/682 [00:19<00:00, 35.70it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: annotated_videos/yellow.mp4 

Wall time: 19.8 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [9]:
is_first_frame = True
challenge_output = 'annotated_videos/extra.mp4'
clip2 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video annotated_videos/extra.mp4
[MoviePy] Writing video annotated_videos/extra.mp4


100%|██████████| 251/251 [00:13<00:00, 19.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: annotated_videos/extra.mp4 

Wall time: 15.1 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))